In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
train_data = pd.read_csv('aksharantar_sampled/hin/hin_train.csv')
test_data = pd.read_csv('aksharantar_sampled/hin/hin_test.csv')
valid_data = pd.read_csv('aksharantar_sampled/hin/hin_valid.csv')

# rename columns ['input_seq', 'target_seq']
train_data.columns = ['input_seq', 'target_seq']
test_data.columns = ['input_seq', 'target_seq']
valid_data.columns = ['input_seq', 'target_seq']

In [21]:
train_data.head()

,input_seq,target_seq
0,bindhya,बिन्द्या
1,kirankant,किरणकांत
2,yagyopaveet,यज्ञोपवीत
3,ratania,रटानिया
4,vaganyache,वागण्याचे


In [64]:
# define Lang
class Lang:
	def __init__(self, wordList):
		self.char2index = {'A': 0, 'Z': 1}
		self.char2count = {}
		self.index2char = {0: 'A', 1: 'Z'}
		self.n_chars = 2

		for word in wordList:
			self.addWord(word)

	def addWord(self, word):
		for char in word:
			self.addChar(char)

	def addChar(self, char):
		if char not in self.char2index:
			self.char2index[char] = self.n_chars
			self.char2count[char] = 1
			self.index2char[self.n_chars] = char
			self.n_chars += 1
		else:
			self.char2count[char] += 1

	def encode(self, word):
		embedded = []
		for i in range(len(word)):
			embedded.append([self.char2index[word[i]]])
		return Variable(torch.LongTensor(embedded))

	def one_hot_encode(self, word):
		one_hot = torch.zeros(len(word), self.n_chars)
		for i in range(len(word)):
			one_hot[i][self.char2index[word[i]]] = 1
		return one_hot
	
	def one_hot_encode_char(self, char):
		one_hot = torch.zeros(1, self.n_chars)
		one_hot[0][self.char2index[char]] = 1
		return one_hot
	
	def decode(self, word):
		decoded = ''
		for i in range(len(word)):
			decoded += self.index2char[word[i]]
		return decoded
	
	def decode_one_hot(self, word):
		decoded = ''
		for i in range(len(word)):
			decoded += self.index2char[word[i].argmax().item()]
		return decoded

In [65]:
# create dataset
class AksharantarDataset(Dataset):
	def __init__(self, data, inp_lang, out_lang):
		self.data = data
		self.inp_lang = inp_lang
		self.out_lang = out_lang

	def __len__(self):
		return len(self.data)

	def __getitem__(self, idx):
		if torch.is_tensor(idx):
			idx = idx.tolist()

		inp_seq = self.inp_lang.one_hot_encode(self.data['input_seq'][idx]).unsqueeze(1)
		out_seq = self.out_lang.one_hot_encode(self.data['target_seq'][idx]).unsqueeze(1)

		sample = {'input_seq': inp_seq, 'target_seq': out_seq}
		return sample

In [66]:
inp_lang = Lang(train_data['input_seq'])
out_lang = Lang(train_data['target_seq'])

train_dataset = AksharantarDataset(train_data, inp_lang, out_lang)
test_dataset = AksharantarDataset(test_data, inp_lang, out_lang)
valid_dataset = AksharantarDataset(valid_data, inp_lang, out_lang)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=True)

In [68]:
# create a seq2seq model using 2 RNNs
class Seq2Seq(nn.Module):
	def __init__(self, input_size, hidden_size, output_size, n_layers=1):
		super(Seq2Seq, self).__init__()
		self.input_size = input_size
		self.hidden_size = hidden_size
		self.output_size = output_size
		self.n_layers = n_layers
				
		# encoder and decoder
		self.encoder = nn.RNN(input_size, hidden_size, n_layers)
		self.decoder = nn.RNN(hidden_size, hidden_size, n_layers)

		# linear layer to get output
		self.linear = nn.Linear(hidden_size, output_size)

	def forward(self, input, hidden):
		# encoder
		output, hidden = self.encoder(input, hidden)
		
		# decoder
		output, hidden = self.decoder(output, hidden)
		
		# get output
		output = self.linear(output)
		return output, hidden
	
	def predict(self, input, out_lang):
		out, hidden = self.forward(input, self.init_hidden(1))
		return out_lang.decode_one_hot(out)
	
	def init_hidden(self, batch_size):
		return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

In [69]:
model = Seq2Seq(inp_lang.n_chars, 128, out_lang.n_chars, 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train 1 input
def train(input_variable, target_variable):
	# zero gradients
	optimizer.zero_grad()
	
	# initialize hidden layer
	hidden = model.init_hidden(1)
	
	# get output
	output, hidden = model.forward(input_variable, hidden)
	
	mx_len = min(len(output), len(target_variable))

	# pad output and target_variable with 'Z'
	while(len(output) < len(target_variable)):
		output = torch.cat((output, out_lang.one_hot_encode_char('Z').view(1, 1, -1)), 0)

	while(len(target_variable) < len(output)):
		target_variable = torch.cat((target_variable, out_lang.one_hot_encode_char('Z').view(1, 1, -1)), 0)
	
	# calculate loss 
	loss = criterion(torch.flatten(output[:mx_len], 0, 1), torch.flatten(target_variable[:mx_len], 0, 1).max(1)[1])
		
	# backpropagate
	loss.backward()
	
	# update weights
	optimizer.step()

	return loss.data.item() / len(input_variable)

In [70]:
from tqdm import tqdm

# train 1 epoch
def trainEpoch(epoch):
	losses = []
	for i in tqdm(range(len(train_data))):
		loss = train(inp_lang.one_hot_encode(train_data['input_seq'][i]).unsqueeze(1), out_lang.one_hot_encode(train_data['target_seq'][i]).unsqueeze(1))
		losses.append(loss)
		print('Epoch: %d, Loss: %.4f' % (epoch, np.mean(losses)))

def trainEpochDataLoader(epoch):
	losses = []
	for i_batch, sample_batched in enumerate(train_dataloader):
		loss = train(sample_batched['input_seq'], sample_batched['target_seq'])
		losses.append(loss)
		print('Epoch: %d, Loss: %.4f' % (epoch, np.mean(losses)))

In [72]:
# train 10 epochs
for i in range(10):
	trainEpoch(i)

  0%|          | 20/51199 [00:00<04:21, 195.99it/s]

Epoch: 0, Loss: 0.6048
Epoch: 0, Loss: 0.5328
Epoch: 0, Loss: 0.4815
Epoch: 0, Loss: 0.5075
Epoch: 0, Loss: 0.4880
Epoch: 0, Loss: 0.4528
Epoch: 0, Loss: 0.4538
Epoch: 0, Loss: 0.4704
Epoch: 0, Loss: 0.4556
Epoch: 0, Loss: 0.4780
Epoch: 0, Loss: 0.4642
Epoch: 0, Loss: 0.4579
Epoch: 0, Loss: 0.4481
Epoch: 0, Loss: 0.4503
Epoch: 0, Loss: 0.4844
Epoch: 0, Loss: 0.4716
Epoch: 0, Loss: 0.4694
Epoch: 0, Loss: 0.4821
Epoch: 0, Loss: 0.4841
Epoch: 0, Loss: 0.4857
Epoch: 0, Loss: 0.4929
Epoch: 0, Loss: 0.4868
Epoch: 0, Loss: 0.4867
Epoch: 0, Loss: 0.4833
Epoch: 0, Loss: 0.4903
Epoch: 0, Loss: 0.4884
Epoch: 0, Loss: 0.4875
Epoch: 0, Loss: 0.5122
Epoch: 0, Loss: 0.5152
Epoch: 0, Loss: 0.5148
Epoch: 0, Loss: 0.5099
Epoch: 0, Loss: 0.5073
Epoch: 0, Loss: 0.5070
Epoch: 0, Loss: 0.5103
Epoch: 0, Loss: 0.5073
Epoch: 0, Loss: 0.5040
Epoch: 0, Loss: 0.5021
Epoch: 0, Loss: 0.5031
Epoch: 0, Loss: 0.4992


  0%|          | 68/51199 [00:00<03:50, 222.16it/s]

Epoch: 0, Loss: 0.4991
Epoch: 0, Loss: 0.4969
Epoch: 0, Loss: 0.4925
Epoch: 0, Loss: 0.5127
Epoch: 0, Loss: 0.5117
Epoch: 0, Loss: 0.5128
Epoch: 0, Loss: 0.5090
Epoch: 0, Loss: 0.5104
Epoch: 0, Loss: 0.5089
Epoch: 0, Loss: 0.5082
Epoch: 0, Loss: 0.5037
Epoch: 0, Loss: 0.5036
Epoch: 0, Loss: 0.5047
Epoch: 0, Loss: 0.5033
Epoch: 0, Loss: 0.5012
Epoch: 0, Loss: 0.4968
Epoch: 0, Loss: 0.5049
Epoch: 0, Loss: 0.5018
Epoch: 0, Loss: 0.4984
Epoch: 0, Loss: 0.5001
Epoch: 0, Loss: 0.4973
Epoch: 0, Loss: 0.4957
Epoch: 0, Loss: 0.4937
Epoch: 0, Loss: 0.4929
Epoch: 0, Loss: 0.4888
Epoch: 0, Loss: 0.4898
Epoch: 0, Loss: 0.4903
Epoch: 0, Loss: 0.4874
Epoch: 0, Loss: 0.4847
Epoch: 0, Loss: 0.4998
Epoch: 0, Loss: 0.4983
Epoch: 0, Loss: 0.4972
Epoch: 0, Loss: 0.4949
Epoch: 0, Loss: 0.4936
Epoch: 0, Loss: 0.4916
Epoch: 0, Loss: 0.4898
Epoch: 0, Loss: 0.4897
Epoch: 0, Loss: 0.4906
Epoch: 0, Loss: 0.4889
Epoch: 0, Loss: 0.4876
Epoch: 0, Loss: 0.4863
Epoch: 0, Loss: 0.4859
Epoch: 0, Loss: 0.4877
Epoch: 0, L

  0%|          | 117/51199 [00:00<03:41, 230.20it/s]

...

  1%|          | 435/51199 [00:02<03:55, 215.44it/s]

Epoch: 0, Loss: 0.4533
Epoch: 0, Loss: 0.4533
Epoch: 0, Loss: 0.4538
Epoch: 0, Loss: 0.4537
Epoch: 0, Loss: 0.4535
Epoch: 0, Loss: 0.4533
Epoch: 0, Loss: 0.4527
Epoch: 0, Loss: 0.4524
Epoch: 0, Loss: 0.4519
Epoch: 0, Loss: 0.4518
Epoch: 0, Loss: 0.4514
Epoch: 0, Loss: 0.4511
Epoch: 0, Loss: 0.4515
Epoch: 0, Loss: 0.4516
Epoch: 0, Loss: 0.4512
Epoch: 0, Loss: 0.4511
Epoch: 0, Loss: 0.4509
Epoch: 0, Loss: 0.4511
Epoch: 0, Loss: 0.4511
Epoch: 0, Loss: 0.4509
Epoch: 0, Loss: 0.4510
Epoch: 0, Loss: 0.4507
Epoch: 0, Loss: 0.4503
Epoch: 0, Loss: 0.4506
Epoch: 0, Loss: 0.4510
Epoch: 0, Loss: 0.4508
Epoch: 0, Loss: 0.4507
Epoch: 0, Loss: 0.4506
Epoch: 0, Loss: 0.4508
Epoch: 0, Loss: 0.4506
Epoch: 0, Loss: 0.4507
Epoch: 0, Loss: 0.4501
Epoch: 0, Loss: 0.4499
Epoch: 0, Loss: 0.4497
Epoch: 0, Loss: 0.4493
Epoch: 0, Loss: 0.4501
Epoch: 0, Loss: 0.4499
Epoch: 0, Loss: 0.4497
Epoch: 0, Loss: 0.4493
Epoch: 0, Loss: 0.4487


  1%|          | 463/51199 [00:02<04:02, 209.19it/s]

Epoch: 0, Loss: 0.4484
Epoch: 0, Loss: 0.4481
Epoch: 0, Loss: 0.4480
Epoch: 0, Loss: 0.4482
Epoch: 0, Loss: 0.4482
Epoch: 0, Loss: 0.4482
Epoch: 0, Loss: 0.4491
Epoch: 0, Loss: 0.4491
Epoch: 0, Loss: 0.4491
Epoch: 0, Loss: 0.4490
Epoch: 0, Loss: 0.4488
Epoch: 0, Loss: 0.4484
Epoch: 0, Loss: 0.4489
Epoch: 0, Loss: 0.4484
Epoch: 0, Loss: 0.4480
Epoch: 0, Loss: 0.4478
Epoch: 0, Loss: 0.4476
Epoch: 0, Loss: 0.4475


KeyboardInterrupt: 

In [18]:
out, hid = model.forward(eng.one_hot_encode('dhruv').unsqueeze(1), model.init_hidden(1))

hin.decode_one_hot(out)

'धुरुव'